In [16]:
import pandas as pd

In [21]:
#Reading the excel file
xls = pd.ExcelFile('Cost Calculation.xlsx')

In [22]:
inVarSheet = pd.read_excel(xls, 'Input Variable', index_col= 0, skiprows=[0])
print(inVarSheet)

                                Not Optimized  Optimized
Instance Hours                          11671     9735.0
Storage Volume                            512        NaN
Percentage of Storage Used                0.9        NaN
Number of snapshot                          7        NaN
Data Transfer                          10.755        NaN
Game Session                             3500        NaN
Number of Request                         NIL     3833.0
Average Time to execute request           NIL     3000.0


In [23]:
fixVarSheet = pd.read_excel(xls, 'Fixed Variable', index_col= 0)

#Storing Rates as variables
ec2Cost = fixVarSheet.loc["EC2 Instance Cost","Rate"]
ebsRates = fixVarSheet.loc["Amazon EBS Rates","Rate"]
snapshotRates = fixVarSheet.loc["Amazon Snapshot Rates","Rate"]
dataTransferIn = fixVarSheet.loc["Data Transfer (In)", "Rate"]
dataTrasferOut = fixVarSheet.loc["Data Transfer (Out)", "Rate"].split(',')
lambdaRequestRates = fixVarSheet.loc["AWS Lambda request", "Rate"]
lambdaComputeRates = fixVarSheet.loc["AWS Lambda compute time ", "Rate"]

print(fixVarSheet)

                                Rate
Fixed Variable                      
EC2 Instance Cost              0.506
Amazon EBS Rates               0.096
Amazon Snapshot Rates           0.05
Data Transfer (In)              Free
Data Transfer (Out)       0.12,0.085
AWS Lambda request               0.2
AWS Lambda compute time     0.000002


In [5]:
# print(ec2Cost)
# print(ebsRates)
# print(snapshotRates)
# print(dataTransferIn)
# print(dataTrasferOut)
# print(lambdaRequestRates)
# print(lambdaComputeRates)

0.506
0.096
0.05
Free
['0.12', '0.085']
0.2
2.1e-06


In [6]:
# Calculating EC2 Cost
## Variables
optimizedInstanceHours = inVarSheet.loc["Instance Hours","Optimized"]
nonOptimizedInstanceHours = inVarSheet.loc["Instance Hours","Not Optimized"]

ebsVolumeSize = inVarSheet.loc["Storage Volume","Not Optimized"]

##Calculation
optimizedHoursCost = optimizedInstanceHours * ec2Cost  ###
nonOptimizedHoursCost = nonOptimizedInstanceHours * ec2Cost  ###

optimizedInstanceMonth = optimizedInstanceHours / 730
nonOptimizedInstanceMonth = nonOptimizedInstanceHours / 730

optimizedVolumeCost = optimizedInstanceMonth * ebsVolumeSize * ebsRates  ###
nonOptimizedVolumeCost = nonOptimizedInstanceMonth *ebsVolumeSize * ebsRates  ###

# print(optimizedInstanceMonth)
# print(optimizedVolumeCost)
# print(nonOptimizedVolumeCost)

13.335616438356164
655.4722191780821
785.8260164383562


In [25]:
#Calculating AMI Cost
## Variables
noOfSnpshot = inVarSheet.loc["Number of snapshot","Not Optimized"]
averageSnpshotSize = ebsVolumeSize * inVarSheet.loc["Percentage of Storage Used","Not Optimized"]
dataTransfer = inVarSheet.loc["Data Transfer","Not Optimized"]
gameSession = inVarSheet.loc["Game Session","Not Optimized"]

## Calculation
snpShotCost = noOfSnpshot * averageSnpshotSize * snapshotRates

totalDataTransfer = dataTransfer * gameSession ##Fixed variable 
dataTransferCost = 9999*float(dataTrasferOut[0]) + (totalDataTransfer - 9999) * float(dataTrasferOut[1]) ## need to add if statement


In [26]:
#Calculating Lambda $ Cloudwatch
## Variable
lambdaRequest = inVarSheet.loc["Number of Request","Optimized"]
lambdaExecuteTime = inVarSheet.loc["Average Time to execute request","Optimized"]

if lambdaRequest < 1000000:
    lambdaRequestCost = lambdaRequestRates
else:
    lambdaRequestCost = lambdaRequest//1000000 * lambdaRequestRates

lambdaExecuteCost = lambdaRequest * lambdaExecuteTime//1000 * lambdaComputeRates


In [28]:
#Preparing the variable
#https://stackoverflow.com/questions/58326392/how-to-create-excel-table-with-pandas-to-excel
totalNonOptCost = nonOptimizedHoursCost + nonOptimizedVolumeCost + snpShotCost + dataTransferCost 
totalOptCost = optimizedHoursCost + optimizedVolumeCost + snpShotCost + dataTransferCost + lambdaRequestCost + lambdaExecuteCost

row1 = f"All Games split into AMI ({ebsVolumeSize}GB)"
row2 = f"Service,Rate,Not Optimized,Optimized"
row3 = f"Instances"
row4 = f"AWS EC2 Instance (g4dn.xlarge),${ec2Cost},{nonOptimizedInstanceHours}hrs x ${ec2Cost} = ${nonOptimizedHoursCost},{optimizedInstanceHours}hrs x ${ec2Cost} = ${optimizedHoursCost}"
row5 = f"Amazon EBS (Volume),${ebsRates},{nonOptimizedInstanceHours} / 730 = {round(nonOptimizedInstanceMonth,2)} instance months\n{ebsVolumeSize}GB x {round(nonOptimizedInstanceMonth,2)} instance months x {ebsRates} = {round(nonOptimizedVolumeCost,2)},{optimizedInstanceHours} / 730 = {round(optimizedInstanceMonth,2)} instance months\n{ebsVolumeSize}GB x {round(optimizedInstanceMonth,3)} instance months x {ebsRates} = {round(optimizedVolumeCost,2)}"
row6 = f"AMI"
row7 = f"Amazon EBS (Snapshot),{snapshotRates} per GB-month of data stored,{noOfSnpshot} Snapshot x {averageSnpshotSize} GB x ${snapshotRates} = ${snpShotCost}"
row8 = f"Data Transfer (In),Free,$0"
row9 = f"Data Transfer (Out),${dataTrasferOut[0]} per GB-month for next 9.999TB/month\n${dataTrasferOut[1]}per GB for next 40TB/month,${round(dataTransferCost,2)}"
row10 = f"Lambda & Cloudwatch"
row11 = f"AWS Lambda request,${lambdaRequestRates} per 1M request,$0(Not Used),${lambdaRequestCost}(under 1M request)"
row12 = f"AWS Lamdba compute time,${lambdaComputeRates} per 1000ms,$0(Not Used),${lambdaRequest} x {lambdaExecuteTime}ms x ${lambdaComputeRates} = ${round(lambdaExecuteCost,2)}"
row13 = f"Amazon CloudWatch Events,Free,$0(Not Used),$0"
row14 = f",Total,${round(totalNonOptCost,2)},${round(totalOptCost,2)}"

tableData = [row1.split(','),row2.split(','),row3.split(','),row4.split(','),row5.split(','),row6.split(','),row7.split(','),row8.split(','),row9.split(','),row10.split(','),row11.split(','),row12.split(','),row13.split(','),row14.split(',')]
df = pd.DataFrame(tableData)
df.to_csv('toBeLatex.csv')

In [12]:
# #Preparing the variable
# #https://stackoverflow.com/questions/58326392/how-to-create-excel-table-with-pandas-to-excel
# totalNonOptCost = nonOptimizedHoursCost + nonOptimizedVolumeCost + snpShotCost + dataTransferCost 
# totalOptCost = optimizedHoursCost + optimizedVolumeCost + snpShotCost + dataTransferCost + lambdaRequestCost + lambdaExecuteCost

# column1 = f"All Games split into AMI ({ebsVolumeSize}GB),Service,Instances,AWS EC2 Instance (g4dn.xlarge),Amazon EBS (Volume),AMI,Amazon EBS (Snapshot),Data Transfer (In),Data Transfer (Out),Lambda & Cloudwatch,AWS Lambda request,AWS Lamdba compute time,Amazon CloudWatch Events,Total"
# column2 = f",Rate,,${ec2Cost},${ebsRates},${snapshotRates} per GB-month of data stored,Free,${dataTrasferOut[0]} per GB-month for next 9.999TB/month\n${dataTrasferOut[1]}per GB for next 40TB/month,,${lambdaRequestRates} per 1M request,${lambdaComputeRates} per 1000ms,Free,"
# column3 = f",Not Optimized,{nonOptimizedInstanceHours}hrs x ${ec2Cost} = ${nonOptimizedHoursCost},{nonOptimizedInstanceHours} / 730 = {round(nonOptimizedInstanceMonth,2)} instance months\n{ebsVolumeSize}GB x {round(nonOptimizedInstanceMonth,2)} instance months x {ebsRates} = {round(nonOptimizedVolumeCost,2)},,{noOfSnpshot} Snapshot x {averageSnpshotSize} GB x ${snapshotRates} = ${snpShotCost},$0,${round(dataTransferCost,2)},,$0(Not Used),$0(Not Used),$0(Not Used),${round(totalNonOptCost,2)}"
# column4 = f",Optimized,,{optimizedInstanceHours}hrs x ${ec2Cost} = ${optimizedHoursCost},{optimizedInstanceHours} / 730 = {round(optimizedInstanceMonth,2)} instance months\n{ebsVolumeSize}GB x {round(optimizedInstanceMonth,3)} instance months x {ebsRates} = {round(optimizedVolumeCost,2)},,${lambdaRequestCost}(under 1M request),${lambdaRequest} x {lambdaExecuteTime}ms x ${lambdaComputeRates} = ${round(lambdaExecuteCost,2)},$0,${round(totalOptCost,2)}"

# pd.Data


['All Games split into AMI (512GB)', 'Service', 'Instances', 'AWS EC2 Instance (g4dn.xlarge)', 'Amazon EBS (Volume)', 'AMI', 'Amazon EBS (Snapshot)', 'Data Transfer (In)', 'Data Transfer (Out)', 'Lambda & Cloudwatch', 'AWS Lambda request', 'AWS Lamdba compute time', 'Amazon CloudWatch Events', 'Total']
